In [29]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

In [3]:
park = Customer('Park', 100)
park

Customer(name='Park', fidelity=100)

In [5]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    def total(self):
        return self.price * self.quantity

In [14]:
class Order:
  """고객(namedtuple) alc LineItem의 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
  def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

  def total(self):
      """__total 속성이 없으면 전체 계산할 값을 계산"""
      if not hasattr(self, '__total'):
          self.__total = sum(item.total() for item in self.cart)
      return self.__total

  def due(self):
      """할인금액 차감"""
      if self.promotion is None:
          discount = 0
      else:
          discount = self.promotion.discount(self)
      return self.total() - discount

  def __repr__(self):
      fmt = '<Order total: {:.2f} due: {:.2f}>'
      return fmt.format(self.total(), self.due())

In [7]:
class Promotion(ABC):
    """할인 혜택 클래스들의 형태를 선언"""

    @abstractmethod
    def discount(self, order):
        """할인액을 구체적인 숫자로 반환"""
        pass

In [12]:
class FidelityPromo(Promotion):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""

    def discount(self, order):
      return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""

    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

class LargeOrderPromo(Promotion):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""

    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [10]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [
    LineItem('banana', 4, .5),
    LineItem('apple', 10, 1.5),
    LineItem('watermellon', 5, 5.0)
]

In [15]:
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [16]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [17]:
banana_cart = [
    LineItem('banana', 30, .5),
    LineItem('apple', 10, 1.5)
]

In [20]:
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [21]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

In [25]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [27]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [28]:
class Order:
    """고객(namedtuple) 및 LineItem 클래스의 인스턴스들을 cart로 받아서 총 계산할 가격을 산출"""
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [30]:
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

In [32]:
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [34]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [35]:
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [36]:
Order(ann, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [37]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
    """최대로 할인받을 금액을 반환"""
    return max(promo(order) for promo in promos)

In [38]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [39]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [40]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

In [41]:
globals().keys()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', 'ABC', 'abstractmethod', '_i2', 'namedtuple', 'Customer', '_i3', 'park', '_3', '_i4', 'LineItem', '_i5', '_i6', 'Order', '_i7', 'Promotion', '_i8', 'FidelityPromo', 'BulkItemPromo', 'LargeOrderPromo', '_i9', '_i10', 'joe', 'ann', 'cart', '_i11', '_11', '_i12', '_i13', '_13', '_i14', '_i15', '_15', '_i16', '_16', '_i17', 'banana_cart', '_i18', '_18', '_i19', '_19', '_i20', '_20', '_i21', 'long_order', '_i22', '_22', '_i23', '_i24', '_24', '_i25', '_25', '_i26', '_26', '_i27', '_27', '_i28', '_i29', '_i30', 'fidelity_promo', 'bulk_item_promo', 'large_order_promo', '_i31', '_31', '_i32', '_32', '_i33', '_33', '_i34', '_34', '_i35', '_35', '_i36', '_36', '_i37', 'promos', 'best_promo', '_i38', '_38', '_i39', '_39', '_i40', '_40', '_i41'])

In [42]:
globals()['best_promo']

<function __main__.best_promo(order)>

In [43]:
promos = [globals()[name] for name in globals()
            if name.endswith('_promo')
            and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [44]:
def deco (func):
  def inner():
    print('running inner()')
  return inner

In [45]:
@deco
def target():
  print("running target()")

target()

running inner()


In [46]:
target

<function __main__.deco.<locals>.inner()>

In [48]:
registry = []
def register(func):
  print('running register(%s)' % func)
  registry.append(func)
  return func

@register
def f1():
  print('running f1()')

@register
def f2():
  print('running f2()')

def f3():
  print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x79b0e658ab90>)
running register(<function f2 at 0x79b1136fe830>)
running main()
registry -> [<function f1 at 0x79b0e658ab90>, <function f2 at 0x79b1136fe830>]
running f1()
running f2()
running f3()


In [50]:
promos = []

def promotion(promo_func):
  """promotion() decorator는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
  promos.append (promo_func)
  return promo_func

@promotion
def fidelity_promo(order):
  """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
  return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
  "20개 이상의 동일 상품을 구입하면 10% 할인 적용""'"
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * 0.1
  return discount

@promotion
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
      return order.total() * 0.07
    return 0

def best_promo(order):
  """최대로 할인받을 금액을 반환한다."""
  return max(promo(order) for promo in promos)

In [51]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [52]:
b=6
f1(3)

3
6


In [53]:
b = 6
def f2(a):
    print(a)
    print(b)
    b=9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [54]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b=9

f3(3)

3
6


In [55]:
b

9

In [56]:
f3(3)

3
9


In [57]:
b=30
f3(3)

3
30


In [58]:
class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [59]:
avg = Averager()
avg(10)

10.0

In [60]:
avg(11)

10.5

In [61]:
avg(12)

11.0

In [63]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

In [64]:
avg = make_averager()
avg(10)

10.0

In [65]:
avg(11)

10.5

In [66]:
avg(12)

11.0

In [68]:
avg.__code__.co_varnames

('new_value', 'total')

In [69]:
avg.__code__.co_freevars

('series',)

In [70]:
avg.__closure__

(<cell at 0x79b0e68775e0: list object at 0x79b0e659a880>,)

In [71]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [72]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

In [74]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [75]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

In [77]:
avg = make_averager()
avg(10)

10.0

In [80]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [84]:
@clock
def snooze(seconds) :
  time. sleep (seconds)

@clock
def factorial(n):
  return 1 if n < 2 else n*factorial(n-1)

if __name__== '__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze (.123)
  print('*'*40, 'Calling factorial(6)')
  print ('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12333312s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000090s] factorial(1) -> 1
[0.00004020s] factorial(2) -> 2
[0.00006281s] factorial(3) -> 6
[0.00008870s] factorial(4) -> 24
[0.00011445s] factorial(5) -> 120
[0.00014356s] factorial(6) -> 720
6! = 720


In [97]:
import time
import functools

def clock(func):
  @functools.wraps(func)
  def clocked (*args, ** kwargs):
    t0 = time.time()
    result = func(*args, ** kwargs)
    elapsed = time. time () - t0
    arg_lst = []
    name = func.__name__
    if args:
      arg_lst.append(', '.join(repr(arg) for arg in args))
    if kwargs:
      pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
      arg_lst.append(', '.join(pairs))
    arg_str = ', '.join(arg_lst)
    print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result) )
    return result
  return clocked

In [98]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__== '__main__':
  print('*'*40, 'Calling snooze(.123)')
  snooze(.123)
  print('*'*40, 'Calling factorial(6)')
  print ('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12316847s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000143s] factorial(1) -> 1
[0.00041151s] factorial(2) -> 2
[0.00088882s] factorial(3) -> 6
[0.00092864s] factorial(4) -> 24
[0.00095034s] factorial(5) -> 120
[0.00097394s] factorial(6) -> 720
6! = 720
